In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image, ImageFile
import torch.nn.functional as F

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

1. EfficientNet-B0 모델 + Dropout

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torchvision.models import efficientnet_b0
model = efficientnet_b0(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.classifier[1].in_features, 7)
)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 95.7MB/s]


전처리

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20, save_path='model.pth'):
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0
            loop = tqdm(enumerate(dataloader), total=len(dataloader), desc=f'{phase.upper()}')

            for i, (inputs, labels) in loop:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                loop.set_postfix(loss=loss.item())

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), save_path)

    print(f'\nBest Validation Accuracy: {best_acc:.4f}')

1. FER2013 pretraining

In [ ]:
fer_train_dataset = datasets.ImageFolder('/content/drive/MyDrive/data/fer2013/train', transform=train_transforms)
fer_val_dataset = datasets.ImageFolder('/content/drive/MyDrive/data/fer2013/test', transform=val_transforms)
fer_train_loader = DataLoader(fer_train_dataset, batch_size=64, shuffle=True, num_workers=0)
fer_val_loader = DataLoader(fer_val_dataset, batch_size=64, shuffle=False, num_workers=0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
train_model(model,
            criterion,
            optimizer,
            fer_train_loader,
            fer_val_loader,
            num_epochs=20,
            save_path='/content/drive/MyDrive/efficient_fer2013_pretrained.pth')

2. Ydbface 파인튜닝 1차

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/efficient_fer2013_pretrained.pth', map_location=device))
model = model.to(device)

yface_train_dataset = datasets.ImageFolder('/content/drive/MyDrive/data/ydbface/train', transform=train_transforms)
yface_val_dataset = datasets.ImageFolder('/content/drive/MyDrive/data/ydbface/test', transform=val_transforms)
yface_train_loader = DataLoader(yface_train_dataset, batch_size=64, shuffle=True, num_workers=0)
yface_val_loader = DataLoader(yface_val_dataset, batch_size=64, shuffle=False, num_workers=0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train_model(model,
            criterion,
            optimizer,
            yface_train_loader,
            yface_val_loader,
            num_epochs=20,
            save_path='/content/drive/MyDrive/efficient_yfacedb_final.pth')

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/weight/efficient_yfacedb_final.pth', map_location=device))
model = model.to(device)

dartmouth_train_dataset = datasets.ImageFolder('/content/drive/MyDrive/data/dartmouth/train', transform=train_transforms)
dartmouth_val_dataset = datasets.ImageFolder('/content/drive/MyDrive/data/dartmouth/val', transform=val_transforms)
dartmouth_train_loader = DataLoader(dartmouth_train_dataset, batch_size=64, shuffle=True, num_workers=0)
dartmouth_val_loader = DataLoader(dartmouth_val_dataset, batch_size=64, shuffle=False, num_workers=0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5)


[Step 3] Dartmouth Fine-tuning 시작


In [ ]:
train_model(model,
            criterion,
            optimizer,
            dartmouth_train_loader,
            dartmouth_val_loader,
            num_epochs=20,
            save_path='/content/drive/MyDrive/weight/efficient_dartmouth_final.pth')


Epoch 1/20
----------


TRAIN: 100%|██████████| 70/70 [22:52<00:00, 19.61s/it, loss=0.79]


train Loss: 1.0299 Acc: 0.6165


VAL:   0%|          | 0/20 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/dartmouth/val/Pleased/S04_13yoM_S04_13yoM_Pleased_left 1.jpg'

In [ ]:
print(dartmouth_train_dataset.class_to_idx)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


적용하는 inference 함수

In [ ]:
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'pleased', 'sad', 'surprise']

def predict_emotion(image_path, model, transform, device):
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        probs = F.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        pred_label = emotion_labels[pred_idx]
        confidence = probs[0][pred_idx].item()
    return pred_label, confidence

In [ ]:
test_image_path = '/content/drive/MyDrive/data/dartmouth_combined/happy/S25_13yoF_Happy_right_0953.JPG'
model.load_state_dict(torch.load('/content/drive/MyDrive/efficient_dartmouth_final.pth', map_location=device))
model = model.to(device)
label, prob = predict_emotion(test_image_path, model, val_transforms, device)
print(f"Predicted emotion: {label} ({prob*100:.2f}%)")